# IMPORT & SPLIT

In [ ]:
using CSV, DataFrames, Statistics, Dates, Gadfly, LinearAlgebra, Distributions, Random, ScikitLearn, GLM, Plots

## Fonctions globales

#### Fonction générique pour former un dataframe contenant les variables explicatives
- Array of data contient les dataframes des va explicatives
- list_of_va contient le type des données i.e ["sum" "max" ...]
- surverse contient les données de surverse

In [ ]:
function createDataEx(array_of_data, list_of_va, dates)
    df = DataFrame(date = dates)
    
    for va in 1:length(list_of_va)
        array = array_of_data[va]
        McTavish = Array{Union{Missing, Int64}}(missing, size(df,1))
        Bellevue = Array{Union{Missing, Int64}}(missing, size(df,1))
        Assomption = Array{Union{Missing, Int64}}(missing, size(df,1))
        Trudeau = Array{Union{Missing, Int64}}(missing, size(df,1))
        StHubert = Array{Union{Missing, Int64}}(missing, size(df,1))
        
        for i=1:size(df,1)
            ind = findfirst(array[:,:date] .== df[i,:date])
            McTavish[i] = array[ind,:McTavish]
            Bellevue[i] = array[ind,:Bellevue]
            Assomption[i] = array[ind,:Assomption]
            Trudeau[i] = array[ind,:Trudeau]
            StHubert[i] = array[ind,:StHubert]
        end
        
        df[!,Symbol(list_of_va[va] * "McTavish")] = McTavish
        df[!,Symbol(list_of_va[va] * "Bellevue")] = Bellevue   
        df[!,Symbol(list_of_va[va] * "Assomption")] = Assomption   
        df[!,Symbol(list_of_va[va] * "Trudeau")] = Trudeau   
        df[!,Symbol(list_of_va[va] * "StHubert")] = StHubert
    end
    
    return df
end



#### Fonction pour recuperer seulement les donnees d'un ouvrage

In [ ]:
function getOuvrage(data, ouvrage)
    return filter(row -> row.NO_OUVRAGE == ouvrage, data)
end

#### Fonction pour recuperer seulement les donnees contenu aux bonnes dates

In [ ]:
function getDataFromDates(df, dates)
    return filter(row -> row.date in dates, df)
end



### Fonction pour la conversion d'un dataframe en matrix/array
- On ignore les dates et les ouvrages si necessaire
- les modeles scikit requierent des arrays et non des dataframes, alors cette fonction nous permettera des les utiliser plus aisément

In [ ]:
function toArray(df::DataFrame)
    
    if :NO_OUVRAGE in names(df)
        return convert(Matrix, df[:, Not([:date, :NO_OUVRAGE])])
    elseif :date in names(df)
        return convert(Matrix, df[:, Not(:date)])
    else
        return convert(Matrix, df)
    end
end
function toArray(df::DataFrameRow)
    
    if :NO_OUVRAGE in names(df)
        return convert(Vector, df[Not([:date, :NO_OUVRAGE])])
    elseif :date in names(df)
        return convert(Vector, df[Not(:date)])
    else
        return convert(Vector, df)
    end
end 

In [ ]:
"""
    splitdataframe(df::DataFrame, p::Real)

Partitionne en un ensemble d'entraînement et un ensemble de validation un DataFrame.

### Arguments
- `df::DataFrame` : Un DataFrame
- `p::Real` : La proportion (entre 0 et 1) de données dans l'ensemble d'entraînement.

### Détails

La fonction renvoie deux DataFrames, un pour l'ensemble d'entraînement et l'autre pour l'ensemble de validation.

### Exemple

\```
 julia> splitdataframe(df, p.7)
\```

"""
function splitdataframe(df::DataFrame, p::Real)
   @assert 0 <= p <= 1 
    
    n = size(df,1)
    
    ind = shuffle(1:n)
    
    threshold = Int64(round(n*p))
    
    indTrain = sort(ind[1:threshold])
    
    indTest = setdiff(1:n,indTrain)
    
    dfTrain = df[indTrain,:]
    dfTest = df[indTest,:]
    
    return dfTrain, dfTest
    
end

## Lecture des fichiers des variables explicatives et des surverses

In [ ]:
x_train_max = CSV.read("x_train_max.csv");
x_pred_max = CSV.read("x_pred_max.csv");

In [ ]:
x_train_sum = CSV.read("x_train_sum.csv");
x_pred_sum = CSV.read("x_pred_sum.csv");

In [ ]:
x_sum_last_2 = CSV.read("sum_day_last_2.csv")
x_train_sum_last_2 = filter(row -> Year(row[:date]) != Year(2019), x_sum_last_2)
x_pred_sum_last_2  = filter(row -> Year(row[:date]) == Year(2019), x_sum_last_2);

In [ ]:
x_max_2 = CSV.read("maxPrecBy2hours.csv")
x_train_max_2 = filter(row -> Year(row[:date]) != Year(2019), x_max_2)
x_pred_max_2  = filter(row -> Year(row[:date]) == Year(2019), x_max_2);

In [ ]:
x_max_3 = CSV.read("maxPrecBy3hours.csv")
x_train_max_3 = filter(row -> Year(row[:date]) != Year(2019), x_max_3)
x_pred_max_3  = filter(row -> Year(row[:date]) == Year(2019), x_max_3);

In [ ]:
x_max_4 = CSV.read("maxPrecBy4hours.csv")
x_train_max_4 = filter(row -> Year(row[:date]) != Year(2019), x_max_4)
x_pred_max_4  = filter(row -> Year(row[:date]) == Year(2019), x_max_4);

In [ ]:
x_max_6 = CSV.read("maxPrecBy6hours.csv")
x_train_max_6 = filter(row -> Year(row[:date]) != Year(2019), x_max_6)
x_pred_max_6  = filter(row -> Year(row[:date]) == Year(2019), x_max_6);

In [ ]:
x_max_8 = CSV.read("maxPrecBy8hours.csv")
x_train_max_8 = filter(row -> Year(row[:date]) != Year(2019), x_max_8)
x_pred_max_8  = filter(row -> Year(row[:date]) == Year(2019), x_max_8);

In [ ]:
x_max_12 = CSV.read("maxPrecBy12hours.csv")
x_train_max_12 = filter(row -> Year(row[:date]) != Year(2019), x_max_12)
x_pred_max_12  = filter(row -> Year(row[:date]) == Year(2019), x_max_12);

### Décision des variables explicatives considérées pour les régressions

Suite à nos tests nous avons décidés de ne pas prendre les max des sommes des 6, 8 et 12 heures

In [ ]:
y_train = CSV.read("surverse_list.csv");
x_train = createDataEx(
    [
        x_train_max,
        x_train_sum,
        x_train_sum_last_2,
        x_train_max_2,
        x_train_max_3,
        x_train_max_4,
    ],
    [
        "max",
        "sum",
        "sum2",
        "max2",
        "max3",
        "max4",
    ], x_train_max_2[:, :date]
)

x_pred = createDataEx(
    [
        x_pred_max,
        x_pred_sum,
        x_pred_sum_last_2,
        x_pred_max_2,
        x_pred_max_3,
        x_pred_max_4,
    ],
    [
        "max",
        "sum",
        "sum2",
        "max2",
        "max3",
        "max4",
    ], x_pred_max_2[:, :date]
)
dropmissing!(x_pred)

#### Ouvrages observés

In [ ]:
ouvrages = ["3260-01D", "3350-07D", "4240-01D", "4350-01D", "4380-01D"]

## Analyse sur nos données

In [ ]:
histogram(convert(Matrix, x_train[:, Not(:date)]))

#### on peut voir que les données sont reparties sur de très grandes distances et très peu uniformément
- nous allons donc tenter d'avoir une répartition des données plus intéressante a l'aide d'une fonction logarithmique de la forme
$ f(x) = ln(x+1) $

In [ ]:
x_values = DataFrame([log.(col.+1) for col = eachcol(x_train[:, Not(:date)])])
display(histogram(convert(Matrix, x_values)))
display(describe(x_values, :mean, :std, :min, :q25, :median, :q75, :max))
x_train_log = log.(x_train[:, Not(:date)] .+ 1)
x_pred_log = log.(x_pred[:, Not(:date)] .+ 1)
x_train_log.date = x_train.date
x_pred_log.date = x_pred.date;

#### On constate qu'avec cette fonction nous obtenons une réparition beaucoup plus intéressante

## Multicolinéarité

In [ ]:
CUTOFF_PCA = 10
@sk_import decomposition : PCA

### Analyse des composantes principales
- L'analyse par les composantes principales est une bonne manière de réduire la multicolinéarité ce qui est important pour les modèles qui n'en font pas abstraction par eux-même

In [ ]:
# analyse en composantes principales pour les données non modifiées
pca = PCA()
x_train_matrix = pca.fit_transform(toArray(x_train))
explained_variance = cumsum(pca.explained_variance_ratio_)
display(Plots.plot(
    1:length(explained_variance),
    explained_variance,
    ylims=(0, 1.0),
    xlabel="Number of components",
    ylabel="Cumulative sum of explained Variance"
))

x_train_pca = DataFrame(
    date=x_train[!,:date],
    x₁=x_train_matrix[:,1],
    x₂=x_train_matrix[:,2],
    x₃=x_train_matrix[:,3],
    x₄=x_train_matrix[:,4],
    x₅=x_train_matrix[:,5],
    x₆=x_train_matrix[:,6],
    x₇=x_train_matrix[:,7],
    x₈=x_train_matrix[:,8],
    x₉=x_train_matrix[:,9],
    x₁₀=x_train_matrix[:,10],
);

In [ ]:
# analyse en composantes principales pour les données mise au logarithme
pca = PCA()
x_train_log_matrix = pca.fit_transform(toArray(x_train_log))
explained_variance = cumsum(pca.explained_variance_ratio_)
display(Plots.plot(
    1:length(explained_variance),
    explained_variance,
    ylims=(0, 1.0),
    xlabel="Number of components",
    ylabel="Cumulative sum of explained Variance",
    title="With ln(1+x)"
))

x_train_log_pca = DataFrame(
    date=x_train[!,:date],
    x₁=x_train_log_matrix[:,1],
    x₂=x_train_log_matrix[:,2],
    x₃=x_train_log_matrix[:,3],
    x₄=x_train_log_matrix[:,4],
    x₅=x_train_log_matrix[:,5],
    x₆=x_train_log_matrix[:,6],
    x₇=x_train_log_matrix[:,7],
    x₈=x_train_log_matrix[:,8],
    x₉=x_train_log_matrix[:,9],
    x₁₀=x_train_log_matrix[:,10],
);

##  Séparations des données pour les ensembles d'entrainement et de test

In [ ]:
Y = y_train
X = x_pred
X_PCA = x_train_pca
X_LOG_PCA = x_train_log_pca
X_BASE = x_train
X_BASE_LOG = x_train_log;

In [ ]:
function dataSplit(ouvrage, x_data)
    y = getOuvrage(Y, ouvrage)
    ytrain, ytest = splitdataframe(y, .80)

    xtrain = getDataFromDates(x_data, ytrain.date)
    ytrain = getDataFromDates(ytrain, xtrain.date)

    xtest = getDataFromDates(x_data, ytest.date)
    ytest = getDataFromDates(ytest, xtest.date)
    
    return xtrain, xtest, ytrain, ytest
end

## Modèles envisagés

In [ ]:
@sk_import metrics: f1_score;

### Cette fonction nous permettra d'entrainer tous nos modèles en même temps avec les mêmes données et les comparer en fonction de leur précision noté avec la métrique f1

In [ ]:
function train_on_all_ouvrages(model_contructor::Function, data, cutoff)
    pred = []
    test = []
    for ouvrage in ouvrages
        x_train, x_test, y_train, y_test = dataSplit(ouvrage, data)
        
        x_train = toArray(x_train); x_test = toArray(x_test);
        y_train = toArray(y_train); y_test = toArray(y_test);
        
        x_train = x_train[:, 1:cutoff]
        x_test = x_test[:, 1:cutoff]
        
        model = model_contructor(x_train, y_train)
        
        prediction = model.predict(x_test);
        
        push!(test, y_test)
        push!(pred, prediction)
    end
    
    pred = collect(Iterators.flatten(pred))
    test = collect(Iterators.flatten(test))
    return f1_score(pred,test)
end

## Regression Logistique

In [ ]:
@sk_import linear_model: LogisticRegression
function modeleLogistique(x_train, y_train) 
    model = LogisticRegression(
        penalty="l2",
        solver="liblinear",
        random_state=234,
    )
    model.fit(x_train, y_train);
    return model
end

## Modele de regression bayésienne naive

In [ ]:
@sk_import naive_bayes: GaussianNB
function modeleNaifGauss(xtrain, ytrain)
    model = GaussianNB()
    model.fit(xtrain, ytrain);
    return model
end

## Modele de l'arbre de Decision

In [ ]:
@sk_import tree: DecisionTreeClassifier
function modeleArbreDecisif(x_train, y_train)
    model = DecisionTreeClassifier(
        random_state=234,
    )
    model.fit(x_train, y_train);
    return model
end

## Classificateur Ridge 

Ce style de classification utilise une regression ridge pour traiter le problème. Toutes les variables sont converties en {-1, 1}. Le résultat est 0 si la regression donne un nombre négatif, et 1 si il est positif.

In [ ]:
@sk_import linear_model: RidgeClassifier
function modeleClassificationRidge(x_train, y_train)
    model = RidgeClassifier(
        alpha=.5,
    )
    model.fit(x_train, y_train);
    return model
end

## Entrainement de tous les modèles
- Dans cette section nous allons entrainer nos modèles et les comparer pour choisir le meilleur
- Pour être sur de ne pas entrainer nos modèles à prédire uniquement notre ensemble "test" (overfitting), nous employons une méthode qui entrainera plusieurs fois nos modèles avec différentes coupures de nos données pour les ensembles "train" et "test"
- Cela nous permettra de calculer une moyenne des scores f1 de chaque entrainement et d'avoir une métrique beaucoup plus puissante, car nous pouvons assurer que le modèle testé est flexible et polyvalent 
- Lors de nos itérations, nous allons aussi entrainer nos modèles avec différents types de variables, soit avec et sans l'analyse des composantes principales et avec et sans l'application de la finction logarithmique aux données

#### La fonction ci-dessous nous calculera les moyennes des scores f1

In [ ]:
function get_avg_f1(m, iterations, data, cutoff)
    score=[]
    for i = 1:iterations
        f1 = train_on_all_ouvrages(m, data, cutoff);
        push!(score,f1)
    end
    f1 = mean(score)
    f1 = round(f1, digits=4)
    return f1
end

#### Le code ci-dessous effectue les itérations et les entrainements

In [ ]:
ITERATIONS=20
display("$(ITERATIONS)  iterations")

models = [
    modeleLogistique,
    modeleNaifGauss,
    modeleArbreDecisif,
    modeleClassificationRidge,
];

dataSets = [
    (X_BASE, "no pca without log", "no itr"),
    (X_BASE_LOG, "no pca with log", "no itr"),
    (X_PCA, "pca without log", "with itr"),
    (X_LOG_PCA, "pca with log", "with itr")
];

best_f1 = 0
best_model = []

for m in models
    model_f1 = []
    display("======= $(string(m)) ======")
    for data in dataSets
        dataSet = data[1]; data_name = data[2]; is_itr = data[3];
        display("------- $(data_name) ------")
        if is_itr == "with itr"
            for pca_cutoff = 1:CUTOFF_PCA
                f1 = get_avg_f1(m, ITERATIONS, dataSet, pca_cutoff)
                push!(model_f1, f1)
                display(f1)

                if f1 > best_f1
                    best_f1 = f1
                    best_model = [f1, string(m), data_name, pca_cutoff]
                end
            end
        else
            n_cols = size(dataSet[:, Not(:date)], 2)
            f1 = get_avg_f1(m, ITERATIONS, dataSet, n_cols)
                push!(model_f1, f1)
                display(f1)

                if f1 > best_f1
                    best_f1 = f1
                    best_model = [f1, string(m), data_name, n_cols]
            end
        end
    end
    display("MAX f1 for Model $(maximum(model_f1))")
end

display("BEST MODEL IS $(best_model[2]), with f1: $(best_model[1]), dataset: $(best_model[3]), pca cutoff: $(best_model[4])")

**Meilleurs**

Modele logistique .7174 avec pca sans log et cutoff 9

Modele Naif Gauss .6988 avec pca avec log et cutoff 1

## Recreer les meilleurs modèles et les entrainer
- À l'aide du test précédent, nous avons sélectionné les deux meilleurs modèles pour les 2 soumissions au concours et nous allons les réentrainer avec l'ensemble des données que nous avons à notre disposition
- cela veut dire que nous ne faisont plus la séparation entre "train" et "test", nous utilisons toutes les données comme ensemble "train" puisque le kaggle contient l'ensemble "test"

In [ ]:
function creerModeles(modele::Function, data, cutoff)
    modeles = DataFrame(NO_OUVRAGE = [], modele = [])
    
    for ouvrage in ouvrages
        y_train = getOuvrage(Y, ouvrage)

        x_train = getDataFromDates(data, y_train.date)
        y_train = getDataFromDates(y_train, x_train.date)

        model = modele(toArray(x_train)[:, 1:cutoff], toArray(y_train));

        push!(modeles, [ouvrage, model])
    end
    
    return modeles
end

In [ ]:
# Logisitique
cutoff = 9
data = X_PCA

modelesLogistiques = creerModeles(modeleLogistique, data, cutoff)

In [ ]:
# Naif Bayésien
cutoff = 1
data = X_LOG_PCA

modelesNaifGauss = creerModeles(modeleNaifGauss, data, cutoff)

In [ ]:
# Arbre décisif
cutoff = 8
data = X_PCA

modelesArbreDecisif = creerModeles(modeleArbreDecisif, data, cutoff)

## Formatter les predictions en consequences

In [ ]:
pca = PCA(n_components=9)
x_pred_log_pca = pca.fit_transform(toArray(x_pred))
x_pred_logistique = DataFrame(
    date=x_pred[!,:date],
    x₁=x_pred_log_pca[:,1],
    x₂=x_pred_log_pca[:,2],
    x₃=x_pred_log_pca[:,3],
    x₄=x_pred_log_pca[:,4],
    x₅=x_pred_log_pca[:,5],
    x₆=x_pred_log_pca[:,6],
    x₇=x_pred_log_pca[:,7],
    x₈=x_pred_log_pca[:,8],
    x₉=x_pred_log_pca[:,9],
);

In [ ]:
pca = PCA(n_components=8)
x_pred_arbre_pca = pca.fit_transform(toArray(x_pred))
x_pred_arbre = DataFrame(
    date=x_pred[!,:date],
    x₁=x_pred_arbre_pca[:,1],
    x₂=x_pred_arbre_pca[:,2],
    x₃=x_pred_arbre_pca[:,3],
    x₄=x_pred_arbre_pca[:,4],
    x₅=x_pred_arbre_pca[:,5],
    x₆=x_pred_arbre_pca[:,6],
    x₇=x_pred_arbre_pca[:,7],
    x₈=x_pred_arbre_pca[:,8],
);

In [ ]:
x_pred_log = log.(x_pred[:, Not(:date)] .+ 1)
pca = PCA(n_components=1)
x_pred_log_pca = pca.fit_transform(toArray(x_pred))
x_pred_naif = DataFrame(
    date=x_pred[!,:date],
    x₁=x_pred_log_pca[:,1],
);

## Mise en place des valeurs cherchees

In [ ]:
test = CSV.read("data/test.csv")
df = test[:, [:NO_OUVRAGE, :DATE]]
rename!(df, :DATE=>:date)
X_logistique = join(df, x_pred_logistique, on=:date)
X_naif = join(df, x_pred_naif, on=:date)
X_arbre = join(df, x_pred_arbre, on=:date)

### Prédiction des données
- Cette fonction permet d'utiliser des modèles rapidement
- Elle nous permettera donc d'effectuer les predictions facilement avec 5 modèles (1 par ouvrage) pour la même soumission

In [ ]:
function predictValues(dataframe, modeles)
    predictions = []
    for x in eachrow(dataframe)
        modele = filter(row -> row.NO_OUVRAGE == x.NO_OUVRAGE, modeles).modele[1]
        predictions = vcat(predictions, modele.predict([toArray(x)]))
    end
    return round.(predictions)
end

In [ ]:
predictionsLogit = predictValues(X_logistique, modelesLogistiques)

In [ ]:
predictionsNaif = predictValues(X_naif, modelesNaifGauss)

In [ ]:
predictionsArbre = predictValues(X_arbre, modelesArbreDecisif)

### Nombre de valeurs prédites différentes entre les deux modèles sélectionnés

In [ ]:
sum(abs.(predictionsLogit-predictionsNaif))

In [ ]:
sum(abs.(predictionsLogit-predictionsArbre))

# Création du fichier de prédictions pour soumettre sur Kaggle

Dans ce cas-ci, nous prédirons une surverse avec une prediction logistique

#### Création du fichier sampleSubmission.csv pour soumettre sur Kaggle avec modèle naif bayésien

In [ ]:
ID = test[:,:NO_OUVRAGE].*"_".*string.(test[:,:DATE])
sampleSubmission = DataFrame(ID = ID, Surverse=predictionsNaif)
CSV.write("submission_naif.csv",sampleSubmission)

#### Création du fichier sampleSubmission.csv pour soumettre sur Kaggle avec modèle logistique

In [ ]:
ID = test[:,:NO_OUVRAGE].*"_".*string.(test[:,:DATE])
sampleSubmission = DataFrame(ID = ID, Surverse=predictionsLogit)
CSV.write("submission_logit.csv",sampleSubmission)

#### Création du fichier sampleSubmission.csv pour soumettre sur Kaggle avec modèle arbre décisif

In [ ]:
ID = test[:,:NO_OUVRAGE].*"_".*string.(test[:,:DATE])
sampleSubmission = DataFrame(ID = ID, Surverse=predictionsArbre)
CSV.write("submission_logit.csv",sampleSubmission)